In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.colors
from matplotlib import cm

In [ ]:
filepath = ' '  # edit this!!
filename = ' '  # add file name!

In [ ]:
os.chdir(filepath)
data = rasterio.open(filename)

In [ ]:
blues = cm.get_cmap('Blues', 256)
newcolors = blues(np.linspace(0, 1.5, 256))
pink = np.array([248/256, 24/256, 148/256, 1])
white = np.array([1,1,1,0])
newcolors[:10, :] = white
newcmp = matplotlib.colors.ListedColormap(newcolors)
 
kwargs = {"cmap": newcmp, "vmax": 1}

fig, ax = plt.subplots(figsize=(15, 10))
swe_plot = show((data,1),ax=ax,vmin=0.1,**kwargs)
colorbar = fig.colorbar(swe_plot,ax=ax)
ax.set_xlabel('easting [m]')
ax.set_ylabel('northing [m]')
fig.colorbar(cm.ScalarMappable(norm=None, cmap=newcmp), ax=ax)
plt.show()

In [ ]:
import h5py
import rasterio as rio
import xarray as xr

#change this to the folder where all your data are
fp = 'c:/users/marianne/documents/data/'
#max_day is an array with the day of water year that had the highest average swe for that year
max_day = np.load(fp+'max_day.npy') 
daily_sum = np.load(fp + 'daily_swe_sum.npy')
daily_mean = np.load(fp + 'daily_swe_mean.npy')

max_swe_map = {}
i=0
for yr in range(1985,2016):
    hf = h5py.File(fp + 'reanalysis/h5/SN_SWE_WY'+ str(yr) + '.h5', 'r')
    swe = hf.get('SWE')
    idx = max_day[i]
    i=i+1
    tmp = swe[idx,:,:]
    max_swe_map[yr] = tmp
    data = swe[idx,:,:]
    lat = hf.get('lat')
    lon = hf.get('lon')
    data = np.float32(data)
    data[data<0] = np.nan
    rows = lon[:].flatten()
    cols = lat[:].flatten()
    im_save = xr.DataArray(data.transpose(), dims=('y', 'x'), 
            coords={'x': rows, 'y': cols})
    im_save.rio.to_raster(fp + 'max_swe'+str(yr)+'.tif')
np.save('data/max_swe_map.npy',max_swe_map)